# Working demonstration of KINS injector using the Census Income dataset

Notebook organisation:
1. [**imports and utility functions**](#imports)
2. [**dataset description, analysis and preprocessing**](#dataset-description,-analysis-and-preprocessing)
3. [**injection**](#injection) (if you are interested only in the injection mechanism skip the other parts)
4. [**training and evaluation**](#training-and-evaluation)

Note: Internet connection is required to download the dataset.
If the files of the Census Income dataset are in folder `data` Internet connection is not required.

<a id='imports'></a>
## Imports and utility functions

Some necessary imports:
- __re__ for regex operations
- __os__ to use other resources in this repository
- __pandas__ for data retrieval and statistics
- __tensorflow__ for neural networks
- __psyki__ for symbolic knowledge injection
- __typing__ for better quality

In [1]:
import os
from sklearn.model_selection import train_test_split
from tensorflow.python.framework.random_seed import set_seed
from tensorflow.python.keras.utils.np_utils import to_categorical
from psyki.logic import Theory
from psyki.logic.prolog import TuProlog
from psyki.ski import Injector

os.getcwd()
from knowledge import PATH as KNOWLEDGE_PATH
from data import SpliceJunction
from utils import create_uneducated_predictor

In [2]:
SPLICE_KNOWLEDGE_FILE = str(KNOWLEDGE_PATH / SpliceJunction.knowledge_file_name)

# Training parameters

SEED = 0
EPOCHS = 100
BATCH_SIZE = 32
VERBOSE = 1

## Dataset description, analysis and preprocessing
(If you are interested only in the injection part you can skip this section)

Download (if not already present) train and test set.
The dataset contains a sequence of 60 nucleotides and a label.
The label can be:
- __ei__: Exon-intron boundary, parts of the DNA sequence that are retained after splicing (donors);
- __ie__: Intron-exon boundary, parts of the DNA sequence that are spliced out (acceptors);
- __n__: Non-splicing boundary, parts of the DNA sequence that are neither retained nor spliced out.

In [3]:
not_processed = SpliceJunction.get_train()

A first look at the dataset:
- 3190 samples in total, 2552 in the training set and 638 in the test set;
- There are three columns, the first one is the label, the second one is the identifier of the sample and the third one is the sequence;
- Classes are reparted as follows:
    - ei: 768
    - ie: 767
    - n: 1655

In [4]:
not_processed

,0,1,2
0,EI,ATRINS-DONOR-521,CCAGCTGCATCACAGGAGGCCAGCGAGCAGGTCTGTTCCAAGGGCC...
1,EI,ATRINS-DONOR-905,AGACCCGCCGGGAGGCGGAGGACCTGCAGGGTGAGCCCCACCGCCC...
2,EI,BABAPOE-DONOR-30,GAGGTGAAGGACGTCCTTCCCCAGGAGCCGGTGAGAAGCGCAGTCG...
3,EI,BABAPOE-DONOR-867,GGGCTGCGTTGCTGGTCACATTCCTGGCAGGTATGGGGCGGGGCTT...
4,EI,BABAPOE-DONOR-2817,GCTCAGCCCCCAGGTCACCCAGGAACTGACGTGAGTGTCCCCATCC...
...,...,...,...
3185,N,ORAHBPSBD-NEG-2881,TCTCTTCCCTTCCCCTCTCTCTTTCTTTCTTTTCTCTCCTCTTCTC...
3186,N,ORAINVOL-NEG-2161,GAGCTCCCAGAGCAGCAAGAGGGCCAGCTGAAGCACCTGGAGAAGC...
3187,N,ORARGIT-NEG-241,TCTCGGGGGCGGCCGGCGCGGCGGGGAGCGGTCCCCGGCCGCGGCC...
3188,N,TARHBB-NEG-541,ATTCTACTTAGTAAACATAATTTCTTGTGCTAGATAACCAAATTAA...


### Data preprocessing

- we discard columns 1 and create 60 new features from column 2;
- we then expand the 60 features into **4 x 60 = 240** new features. We basically create 4 new features for each of the 60 nucleotides. Each of the 4 new features is a binary feature that is 1 if the nucleotide is A, C, G or T respectively (multi-hot encoding). This is done because there are other symbols other than A, C, G and T in the dataset (e.g., the symbol M is used to represent a nucleotide that can be both A or C).

In [5]:
processed_dataset = SpliceJunction.get_processed_dataset(not_processed)
train, test = train_test_split(processed_dataset, test_size=0.5, random_state=SEED, stratify=processed_dataset.iloc[:, -1])
train

,X_30a,X_30c,X_30g,X_30t,X_29a,X_29c,X_29g,X_29t,X_28a,X_28c,...,X28t,X29a,X29c,X29g,X29t,X30a,X30c,X30g,X30t,240
891,1,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,1
900,0,0,0,1,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,1
2753,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,2
1240,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,1
1904,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,2
467,0,0,1,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0
1521,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
1559,0,1,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,1,2


<a id='injection'></a>
## Injection

### Knowledge

In [6]:
knowledge = TuProlog.from_file(SPLICE_KNOWLEDGE_FILE)
theory = Theory(knowledge, train, SpliceJunction.class_mapping)

In [7]:
for rule_name in sorted(set([rule.lhs.predication for rule in theory.formulae])):
    print(rule_name)

class
ei_stop
exon_intron
ie_stop
intron_exon
m_of_n
not_both_zero
pyramidine_rich


We want to make some rules trainable.
We decide to train:
- `class`
- `exon_intron`
- `intron_exon`
- `pyrimidine_rich`

In [8]:
# theory.set_all_formulae_trainable()
theory.set_formulae_trainable(['class', 'exon_intron', 'intron_exon', 'pyrimidine_rich'])

### The actual injection is as simple as that

In [9]:
set_seed(SEED)
# Here we create a fully-connected NN with one hidden layer of 10 neurons
uneducated = create_uneducated_predictor(train.shape[1]-1, 3, [])
injector = Injector.kbann(uneducated)
educated = injector.inject(theory)

In [10]:
educated.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
history_educated = educated.fit(train.iloc[:, :-1], to_categorical(train.iloc[:, -1]), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=VERBOSE)

2023-04-20 17:56:11.949529: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100
50/50 [==============================] - 10s 8ms/step - loss: 2.4951 - accuracy: 0.5636
Epoch 2/100
50/50 [==============================] - 0s 9ms/step - loss: 2.1917 - accuracy: 0.6163
Epoch 3/100
50/50 [==============================] - 1s 12ms/step - loss: 2.0600 - accuracy: 0.6226
Epoch 4/100
50/50 [==============================] - 1s 12ms/step - loss: 1.9440 - accuracy: 0.6232
Epoch 5/100
50/50 [==============================] - 1s 12ms/step - loss: 1.8361 - accuracy: 0.6232
Epoch 6/100
50/50 [==============================] - 1s 12ms/step - loss: 1.7341 - accuracy: 0.6238
Epoch 7/100
50/50 [==============================] - 1s 12ms/step - loss: 1.6367 - accuracy: 0.6257
Epoch 8/100
50/50 [==============================] - 1s 12ms/step - loss: 1.5445 - accuracy: 0.6307
Epoch 9/100
50/50 [==============================] - 1s 12ms/step - loss: 1.4572 - accuracy: 0.6401
Epoch 10/100
50/50 [==============================] - 1s 12ms/step - loss: 1.3724 - accuracy: 0.6445


In [11]:
_, acc = educated.evaluate(test.iloc[:, :-1], to_categorical(test.iloc[:, -1]))
print(f'test set accuracy of the educated predictor: {acc*100:.2f}%')

50/50 [==============================] - 2s 4ms/step - loss: 0.2241 - accuracy: 0.9273
test set accuracy of the educated predictor: 92.73%
